<a href="https://colab.research.google.com/github/quantaosun/literature_spider/blob/main/literaure_spider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import requests
from xml.etree import ElementTree
import pandas as pd
from datetime import datetime

# Set pandas options to display all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Base URL for PubMed E-utilities API
base_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'

# Current date and January 1 of five years ago
current_date = datetime.now().strftime('%Y/%m/%d')
five_years_ago_date = datetime(datetime.now().year - 5, 1, 1).strftime('%Y/%m/%d')

# Your query
query = f'selective kinase inhibitor AND ("{five_years_ago_date}"[PDAT] : "{current_date}"[PDAT])'

# Create an empty DataFrame to store the results
df = pd.DataFrame(columns=['Title', 'URL', 'Authors', 'Journal', 'Publication Year'])

# Send a GET request to the ESearch utility to search for the query in PubMed
response = requests.get(f'{base_url}esearch.fcgi', params={
    'db': 'pubmed',
    'term': query,
    'retmode': 'xml'
})

# If the request is successful, parse the XML response
if response.status_code == 200:
    root = ElementTree.fromstring(response.text)
    # Get the list of PubMed IDs (PMIDs) from the response
    id_list = [id_elem.text for id_elem in root.iter('Id')]

    # For each PMID, send a GET request to the ESummary utility to retrieve the article details
    for pmid in id_list:
        response = requests.get(f'{base_url}esummary.fcgi', params={
            'db': 'pubmed',
            'id': pmid,
            'retmode': 'xml'
        })
        if response.status_code == 200:
            root = ElementTree.fromstring(response.text)
            # Extract the title, URL, authors, journal, and publication year for each article and add them to the DataFrame
            for docsum in root.iter('DocSum'):
                title = docsum.find('Item[@Name="Title"]').text
                url = f'https://pubmed.ncbi.nlm.nih.gov/{pmid}/'
                authors = docsum.find('Item[@Name="Authors"]').text if docsum.find('Item[@Name="Authors"]') is not None else 'N/A'
                journal = docsum.find('Item[@Name="FullJournalName"]').text if docsum.find('Item[@Name="FullJournalName"]') is not None else 'N/A'
                pub_year = docsum.find('Item[@Name="PubDate"]').text.split()[0] if docsum.find('Item[@Name="PubDate"]') is not None else 'N/A'
                df = df.append({'Title': title, 'URL': url, 'Authors': authors, 'Journal': journal, 'Publication Year': pub_year}, ignore_index=True)

# Display the DataFrame
print(df)

# Save the DataFrame as a CSV file
df.to_csv('literature.csv', index=False)


<ipython-input-8-f20c3bab3c53>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Title': title, 'URL': url, 'Authors': authors, 'Journal': journal, 'Publication Year': pub_year}, ignore_index=True)
<ipython-input-8-f20c3bab3c53>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Title': title, 'URL': url, 'Authors': authors, 'Journal': journal, 'Publication Year': pub_year}, ignore_index=True)
<ipython-input-8-f20c3bab3c53>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Title': title, 'URL': url, 'Authors': authors, 'Journal': journal, 'Publication Year': pub_year}, ignore_index=True)
<ipython-input-8-f20c3bab3c53>:52: FutureWarning: The frame.append method is deprecated 

                                                Title  \
0   A new triazolyl-indolo-quinoxaline induces apo...   
1   A Meta-Analysis to Assess the Efficacy of HER2...   
2   Zebrafish as model system for the biological c...   
3   Integrated proteomic and phosphoproteomic data...   
4   Overcoming the therapeutic limitations of EZH2...   
5   Health-related quality of life in treatment-na...   
6   A Novel LC-MS/MS Method for Therapeutic Drug M...   
7   BAX activation in mouse retinal ganglion cells...   
8   Identification of histidine kinase inhibitors ...   
9   Protein-Protein Interfaces as Druggable Target...   
10  A Phase I Study of the DNA-PK Inhibitor Pepose...   
11  Transcriptomics-inferred dynamics of SARS-CoV-...   
12                      Befotertinib: First Approval.   
13  Effect of Tucatinib on Cardiac Repolarization ...   
14  Tyrosine-kinase inhibitor combined with iodine...   
15  Brief Report: Tyrosine Kinase Inhibitors for L...   
16  Choline metabolism underpin

<ipython-input-8-f20c3bab3c53>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Title': title, 'URL': url, 'Authors': authors, 'Journal': journal, 'Publication Year': pub_year}, ignore_index=True)


In [9]:
#@title ### **Import Google Drive**
#@markdown Click in the "Run" buttom to make your Google Drive accessible.
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
